In [ ]:
import torchvision
from torch.utils.data import Dataset, DataLoader

Part 1. Loading the Dataset

In [ ]:
dataset = 

Part 2. Defining the Neural Network Architecture

Part 3. Train the Model

Part 6. Unsupervised Classification